In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [25]:
df=pd.read_csv(r"D:\SNU\4th sem\DESIGN AND ANALYSIS OF ALGORITHMS\CIA2\Bank_Personal_Loan_Modelling.csv")

In [26]:
df.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [27]:
df.drop(['ID','ZIP Code'],axis=1,inplace=True)

In [28]:
df.columns

Index(['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education',
       'Mortgage', 'Personal Loan', 'Securities Account', 'CD Account',
       'Online', 'CreditCard'],
      dtype='object')

In [29]:
df=df[['Age', 'Experience', 'Income','Family','CCAvg',
       'Education', 'Mortgage', 'Securities Account',
       'CD Account', 'Online', 'CreditCard','Personal Loan']]

In [30]:
x= df.iloc[:,:-1].values
y= df.iloc[:, -1].values

In [31]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=0)

In [32]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [33]:
from keras.models import Sequential
from keras.layers import Dense

In [34]:
model=Sequential()
model.add(Dense(units=6,activation="relu"))
model.add(Dense(units=6,activation="relu"))#second hidden layer is created
model.add(Dense(units=1,activation="sigmoid"))#another layer with different activation fn(o/p layer))

In [35]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics="accuracy")

In [36]:
def fitness(position):
    fitnessVal = 0.0
    for i in range(len(position)):
        xi = position[i]
        fitnessVal += (xi * xi) - (10 * math.cos(2 * math.pi * xi)) + 10
    return fitnessVal

In [37]:
def roulette_wheel_selection(p):

    c = np.cumsum(p)
    r = sum(p) * np.random.rand()
    ind = np.argwhere(r <= c)
   
    return ind[0][0]

In [38]:
import copy
def crossover(p1, p2):
#UNIFORM CROSSOVER
  
    c1 = copy.deepcopy(p1)
    c2 = copy.deepcopy(p2)
    alpha = np.random.uniform(0, 1, *(c1['position'].shape))
    c1['position'] = alpha*p1['position'] + (1-alpha)*p2['position']
    c2['position'] = alpha*p2['position'] + (1-alpha)*p1['position']

    return c1, c2

In [39]:
def mutate(c, mu, sigma):
    y = copy.deepcopy(c)
    flag = np.random.rand(*(c['position'].shape)) <= mu 
    ind = np.argwhere(flag)
    y['position'][ind] += sigma * np.random.randn(*ind.shape)
  
    return y

In [40]:
def bounds(c, varmin, varmax):

    c['position'] = np.maximum(c['position'], varmin)
    c['position'] = np.minimum(c['position'], varmax)

In [41]:
def ga(costfunc, num_var, varmin, varmax, maxit, npop, num_children, mu, sigma, beta):
  
 
    population = {}
    for i in range(npop):                                                         
        population[i] = {'position': None, 'cost': None}                           
 
    bestsol = copy.deepcopy(population)
    bestsol_cost = np.inf                                                         

  
    for i in range(npop):
        population[i]['position'] = np.random.uniform(varmin, varmax, num_var)    
        population[i]['cost'] = costfunc(population[i]['position'])

        if population[i]['cost'] < bestsol_cost:                                  
            bestsol = copy.deepcopy(population[i])                                  

  
    bestcost = np.empty(maxit)

    for it in range(maxit):

    
        costs = []
        for i in range(len(population)):
            costs.append(population[i]['cost'])                                       
            costs = np.array(costs)
            avg_cost = np.mean(costs)                                                   
            if avg_cost != 0:
                costs = costs/avg_cost
                probs = np.exp(-beta*costs)                                                 
 
    for _ in range(num_children//2):                                             
                                                                               


      # Roulette wheel selection
        p1 = population[roulette_wheel_selection(probs)]
        p2 = population[roulette_wheel_selection(probs)]

      # crossover two parents
        c1, c2 = crossover(p1, p2)
          
      # Perform mutation
        c1 = mutate(c1, mu, sigma)
        c2 = mutate(c2, mu, sigma)
      
      # Apply bounds
        bounds(c1, varmin, varmax)
        bounds(c2, varmin, varmax)
      
      
        c1['cost'] = costfunc(c1['position'])                                     
      
        if type(bestsol_cost) == float:
             if c1['cost'] < bestsol_cost:                                           
                bestsol_cost = copy.deepcopy(c1)
        else:
            if c1['cost'] < bestsol_cost['cost']:                                  
                bestsol_cost = copy.deepcopy(c1)

      
      
        if c2['cost'] < bestsol_cost['cost']:                                     
            bestsol_cost = copy.deepcopy(c2)

    # Store best cost
    bestcost[it] = bestsol_cost['cost']

   
    print('Iteration {}: Best Cost = {}'. format(it, bestcost[it]))


    out = population
    Bestsol = bestsol
    bestcost = bestcost
    return (out, Bestsol, bestcost)

In [42]:
model.fit(x_train, y_train, batch_size = 30, epochs = 100)

Epoch 1/100
125/125 [==============================] - 2s 3ms/step - loss: 0.4806 - accuracy: 0.9008
Epoch 2/100
125/125 [==============================] - 0s 3ms/step - loss: 0.3607 - accuracy: 0.9008
Epoch 3/100
125/125 [==============================] - 0s 3ms/step - loss: 0.3079 - accuracy: 0.9008
Epoch 4/100
125/125 [==============================] - 0s 3ms/step - loss: 0.2751 - accuracy: 0.9008
Epoch 5/100
125/125 [==============================] - 0s 3ms/step - loss: 0.2478 - accuracy: 0.9008
Epoch 6/100
125/125 [==============================] - 1s 5ms/step - loss: 0.2235 - accuracy: 0.9008
Epoch 7/100
125/125 [==============================] - 0s 3ms/step - loss: 0.2030 - accuracy: 0.9008
Epoch 8/100
125/125 [==============================] - 0s 4ms/step - loss: 0.1864 - accuracy: 0.9008
Epoch 9/100
125/125 [==============================] - 0s 3ms/step - loss: 0.1723 - accuracy: 0.9008
Epoch 10/100
125/125 [==============================] - 0s 3ms/step - loss: 0.1593 - accura

In [44]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
confusion_matrix(y_test, y_pred)

array([[1137,    5],
       [  18,   90]], dtype=int64)

In [45]:
accuracy_score(y_test, y_pred)

0.9816